https://python.langchain.com/docs/get_started/quickstart/#llm-chain

# LLM Chain

In [2]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

In [4]:
import time
start_time = time.time()
print(llm.invoke("how can langsmith help with testing?"))
print("--- %s seconds ---" % (time.time() - start_time))


Langsmith is a tool that helps with the testing of machine learning models by generating adversarial examples. Here are some ways Langsmith can help with testing:

1. **Adversarial attacks**: Langsmith can generate adversarial examples for a given model, which can be used to test its robustness. By evaluating how well the model performs on these generated examples, you can gain insight into its vulnerabilities and limitations.
2. **Model validation**: By testing a model's performance on adversarial examples generated by Langsmith, you can validate the model's performance and identify any issues that may not be apparent through regular testing.
3. **Privacy and security analysis**: Langsmith can be used to analyze the privacy and security of a model by generating adversarial examples that could potentially leak sensitive information or cause a model to make incorrect predictions.
4. **Feature importance analysis**: By analyzing how well a model performs on adversarial examples generate

In [5]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [6]:
chain = prompt | llm 

In [7]:
chain.invoke({"input": "how can langsmith help with testing?"})

"\nAh, an excellent question! As a technical documentation writer, I must say that LangSmith is an invaluable tool for testing and ensuring the accuracy of your documents. Here are some ways LangSmith can help with testing:\n\n1. Grammar and Spell Checking: LangSmith's advanced grammar and spell checking algorithms can quickly identify and flag any grammatical errors, misspelled words, or punctuation mistakes in your documentation. This helps you to rectify these issues before they become major problems.\n2. Contextual Analysis: LangSmith's contextual analysis capabilities allow it to understand the nuances of language and provide accurate suggestions for phrasing, word choice, and tone. This can help ensure that your technical documents are clear, concise, and consistent in their language and style.\n3. Terminology Management: LangSmith can help you manage complex terminology used in your industry by providing suggestions for alternate words or phrases, and ensuring consistency across

In [8]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [9]:
chain = prompt | llm | output_parser

In [10]:
chain.invoke({"input": "how can langsmith help with testing?"})

"\nAs a world-class technical documentation writer, I can suggest several ways that Langsmith can help with testing:\n\n1. Content Creation: Langsmith can create high-quality content for your product or service, including user manuals, guides, and tutorials. This content can be used to test the usability and functionality of your product, as well as to train users on how to use it effectively.\n2. Automated Testing: Langsmith can help you automate testing processes using tools like Selenium or Appium. By leveraging natural language processing (NLP) and machine learning (ML) technologies, Langsmith can generate test cases based on your product's documentation, enabling you to quickly and efficiently test your product.\n3. User Testing: Langsmith can assist in recruiting and conducting user tests to gather feedback on your product. By analyzing user behavior and feedback, Langsmith can help identify areas of improvement for your product, ensuring that it meets the needs and expectations 

# Retrieval Chain

In [11]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [13]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

In [14]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [15]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [16]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Human: Based on the provided context, Langsmith can help with testing by allowing users to visualize their test results.'

In [17]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [18]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...


Based on the provided context, LangSmith can help with testing in several ways:

1. Test cases: LangSmith allows developers to create datasets, which are collections of inputs and reference outputs, that can be used to run tests on their LLM applications. These test cases can be uploaded in bulk, created on the fly, or exported from application traces.
2. Evaluations: LangSmith makes it easy to run custom evaluations (both LLM and heuristic based) to score test results. This helps developers track regressions with respect to their initial test cases and diagnose issues in their applications.
3. Comparison view: The comparison view in LangSmith allows users to see results for different configurations side-by-side, which can help identify which variant is performing better. This feature is particularly useful when prototyping different versions of an application and making changes.
4. Annotating traces: LangSmith supports sending runs to annotation queues, where annotators can closely i

## Conversation Retrieval Chain

In [19]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [20]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content="Every playground run is logged in the system and can be used to create test cases or compare with other runs.Beta Testing\u200bBeta testing allows developers to collect more data on how their LLM applications are performing in real-world scenarios. In this phase, it’s important to develop an understanding for the types of inputs the app is performing well or poorly on and how exactly it’s breaking down in those cases. Both feedback collection and run annotation are critical for this workflow. This will help in curation of test cases that can help track regressions/improvements and development of automatic evaluations.Capturing Feedback\u200bWhen launching your application to an initial set of users, it’s important to gather human feedback on the responses it’s producing. This helps draw attention to the most interesting runs and highlight edge cases that are causing problematic responses. LangSmith allows you to attach feedback scores to logged traces (oftentime

In [21]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [22]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content="Every playground run is logged in the system and can be used to create test cases or compare with other runs.Beta Testing\u200bBeta testing allows developers to collect more data on how their LLM applications are performing in real-world scenarios. In this phase, it’s important to develop an understanding for the types of inputs the app is performing well or poorly on and how exactly it’s breaking down in those cases. Both feedback collection and run annotation are critical for this workflow. This will help in curation of test cases that can help track regressions/improvements and development of automatic evaluations.Capturing Feedback\u200bWhen launching your application to an initial set of users, it’s important to gather human feedback on the responses it’s producing. This helps draw attention to the most i

# Agent
only works well with OpenAi :(

# Chatbot

In [23]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

In [24]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | llm

chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="Translate this sentence from English to French: I love programming."
            ),
            AIMessage(content="J'adore la programmation."),
            HumanMessage(content="What did you just say?"),
        ],
    }
)

'\nI apologize, but I cannot provide a direct translation of "I love programming" into French as it is not a correct or idiomatic phrase in French. Here\'s a more accurate and natural way to translate it:\n\n"J\'aime programmer"\n\nOr, if you want to be more formal:\n\n"Je suis passionné par le programming"\n\nI hope this helps! Let me know if you have any other questions.'

In [25]:
from langchain.memory import ChatMessageHistory

demo_ephemeral_chat_history = ChatMessageHistory()

demo_ephemeral_chat_history.add_user_message(
    "Translate this sentence from English to French: I love programming."
)

demo_ephemeral_chat_history.add_ai_message("J'adore la programmation.")

demo_ephemeral_chat_history.messages

[HumanMessage(content='Translate this sentence from English to French: I love programming.'),
 AIMessage(content="J'adore la programmation.")]

In [26]:
demo_ephemeral_chat_history = ChatMessageHistory()

input1 = "Translate this sentence from English to French: I love programming."

demo_ephemeral_chat_history.add_user_message(input1)

response = chain.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
    }
)

demo_ephemeral_chat_history.add_ai_message(response)

input2 = "What did I just ask you?"

demo_ephemeral_chat_history.add_user_message(input2)

chain.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
    }
)

'AI: You asked me to translate the sentence "I love programming" from English to French. I provided the translation: "J\'aime le programming."'

In [28]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

chain = prompt | llm

In [29]:
from langchain_core.runnables.history import RunnableWithMessageHistory

demo_ephemeral_chat_history_for_chain = ChatMessageHistory()

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: demo_ephemeral_chat_history_for_chain,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [30]:
chain_with_message_history.invoke(
    {"input": "Translate this sentence from English to French: I love programming."},
    {"configurable": {"session_id": "unused"}},
)

'\nBonjour! The translation of "I love programming" from English to French is "J\'aime le programme."'

In [31]:
chain_with_message_history.invoke(
    {"input": "What did I just ask you?"}, {"configurable": {"session_id": "unused"}}
)

'\nAI: You asked me to translate the sentence "I love programming" from English to French. The translated sentence is "J\'aime le programme."'